Imports

In [1]:
import pandas as pd
import pyarrow.parquet as pq
from sklearn.metrics import classification_report
RANDOM_SEED = 42
batch_size = 1000000

In [2]:
target_channels = pd.read_csv('data/target_channels.csv')['target_channels'].tolist()


EL DATASET TIENE 14 MILLONES DE FILAS :o NO CABE EN EL DATAFRAME NORMAL

El dataset tiene 14 millones de filas y no caben en el dataframe, por ello generamos un codigo para obtener filas de forma parcial.

In [3]:
# Initialize variables
batch_parquet = pq.ParquetFile('data/train.parquet')
# Iterate over batches
for batch  in batch_parquet.iter_batches(batch_size):
    # Filter only relevant channels
    batch_df = batch.to_pandas()

    X = batch_df.drop(columns='is_anomaly')  # Exclude non-features
    y = batch_df['is_anomaly']

    # Train the model on this batch
    #clf.partial_fit(X, y, classes=[0, 1])  # Update model incrementally

    # Optionally, free up memory by deleting the chunk after processing
    #del batch_df  # Explicitly delete the chunk to free memory


print(batch_df.head())
batch_df.describe()

         id  channel_1  channel_10  channel_11  channel_12  channel_13  \
0  14000000    0.13791         0.0         0.0    0.201758    0.240750   
1  14000001    0.13791         0.0         0.0    0.201758    0.242309   
2  14000002    0.13791         0.0         0.0    0.201758    0.242309   
3  14000003    0.13791         0.0         0.0    0.201758    0.242309   
4  14000004    0.13791         0.0         0.0    0.201758    0.222033   

   channel_14  channel_15  channel_16  channel_17  ...  telecommand_351  \
0    0.179529    0.105638    0.729993    0.213785  ...              0.0   
1    0.178021    0.105638    0.731832    0.233385  ...              0.0   
2    0.178021    0.105638    0.731832    0.233385  ...              0.0   
3    0.178021    0.105638    0.731832    0.233385  ...              0.0   
4    0.179529    0.105638    0.729993    0.231877  ...              0.0   

   telecommand_352  telecommand_353  telecommand_354  telecommand_36  \
0              0.0              

,id,channel_1,channel_10,channel_11,channel_12,channel_13,channel_14,channel_15,channel_16,channel_17,...,telecommand_351,telecommand_352,telecommand_353,telecommand_354,telecommand_36,telecommand_376,telecommand_38,telecommand_39,telecommand_40,is_anomaly
count,7.283210e+05,728321.000000,7.283210e+05,7.283210e+05,728321.000000,728321.000000,728321.000000,728321.000000,728321.000000,728321.000000,...,728321.0,728321.0,728321.0,728321.000000,728321.000000,728321.0,728321.000000,728321.000000,728321.000000,728321.000000
mean,1.436416e+07,0.137935,1.145818e-07,7.205464e-10,0.200700,0.247814,0.532603,0.278732,0.752611,0.239872,...,0.0,0.0,0.0,0.000004,0.000005,0.0,0.000003,0.000003,0.000003,0.204067
std,2.102483e+05,0.001829,6.416650e-06,3.075356e-07,0.010660,0.010026,0.202405,0.125290,0.022846,0.010843,...,0.0,0.0,0.0,0.002030,0.002344,0.0,0.001657,0.001657,0.001657,0.403018
min,1.400000e+07,0.092172,0.000000e+00,0.000000e+00,0.147169,0.222033,0.175001,0.094044,0.715283,0.213785,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
25%,1.418208e+07,0.137910,0.000000e+00,0.000000e+00,0.192400,0.239190,0.226296,0.106925,0.724477,0.231877,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
50%,1.436416e+07,0.137910,0.000000e+00,0.000000e+00,0.200198,0.246989,0.622610,0.310233,0.757575,0.237907,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
75%,1.454624e+07,0.137910,0.000000e+00,0.000000e+00,0.207996,0.254787,0.627687,0.344895,0.768608,0.246954,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
max,1.472832e+07,0.425061,4.236102e-04,1.342669e-04,0.229832,0.273503,0.971057,0.541334,0.798028,0.271076,...,0.0,0.0,0.0,1.000000,1.000000,0.0,1.000000,1.000000,1.000000,1.000000


Observamos si los tiempos que tenemos que predecir en test, para en que tiempos tenemos que predecir

In [4]:
# Initialize variables
batch_parquet = pq.ParquetFile('data/test.parquet')
train_ids = []
test_ids = []
# Iterate over batches
for batch  in batch_parquet.iter_batches(batch_size):
    # Filter only relevant channels
    batch_df = batch.to_pandas()
    test_ids.extend(batch_df['id'].values)

batch_parquet = pq.ParquetFile('data/train.parquet')
for batch  in batch_parquet.iter_batches(batch_size):
    # Filter only relevant channels
    batch_df = batch.to_pandas()
    train_ids.extend(batch_df['id'].values)

print(f"minimum of train_ids {min(train_ids)}, train_ids {max(train_ids)}")
print(f"minimum of test_ids {min(test_ids)}, maximum {max(test_ids)}")

minimum of train_ids 0, train_ids 14728320
minimum of test_ids 14728321, maximum 15249600


Podemos ver que los datos de test van inmediantemente despues de los de train.

Probamos a realizar un modelo para las predicciones basico usando aeon.

In [4]:
# empezamos usando solo las columnas que segun los creadores mejor funcionan
best_channels = ["channel_41","channel_42","channel_43","channel_44","channel_45","channel_46"]

In [6]:
from aeon.transformations.collection.convolution_based import Rocket
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.utils import shuffle
import pandas as pd

batch_parquet = pq.ParquetFile('data/train.parquet')

# 1. Init global objects
rocket = Rocket(n_kernels=100)
scaler = StandardScaler()
clf = SGDClassifier(loss='log_loss')  # Logistic regression (incremental)

classes = [0, 1]
fitted_rocket = False
fitted_scaler = False

for batch in batch_parquet.iter_batches(batch_size):
    batch_df = batch.to_pandas()
    X = batch_df.drop(columns='is_anomaly')  # Exclude non-features
    X = X[best_channels] # filter for the bests
    y = batch_df['is_anomaly']


    # a) First batch: fit Rocket
    if not fitted_rocket:
        rocket.fit(X)
        fitted_rocket = True

    # b) Transform batch
    X_features = rocket.transform(X)  # (n_samples, n_features)

    # c) Scale
    if not fitted_scaler:
        scaler.fit(X_features)
        fitted_scaler = True
    X_scaled = scaler.transform(X_features)

    # d) Partial fit classifier
    clf.partial_fit(X_scaled, y, classes=classes)

# Evaluate the model
batch_df = next(batch_parquet.iter_batches(batch_size)).to_pandas()
X_val = batch_df.drop(columns='is_anomaly')  # Exclude non-features
X_val = X_val[best_channels] # filter for the bests
y_val = batch_df['is_anomaly']
X_val_transformed = scaler.transform(rocket.transform(X_val))
y_pred = clf.predict(X_val_transformed)


print(classification_report(y_val, y_pred))


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


              precision    recall  f1-score   support

           0       0.87      1.00      0.93    874913
           1       0.00      0.00      0.00    125087

    accuracy                           0.87   1000000
   macro avg       0.44      0.50      0.47   1000000
weighted avg       0.77      0.87      0.82   1000000



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Otra forma de obtener este conjunto pesado de datos y transformarlo a la clase dataset de pytorch

In [11]:
import torch
from torch.utils.data import Dataset
import pyarrow.parquet as pq
import torch.nn as nn
import numpy as np


class TimeSeriesParquetDataset(Dataset):
    def __init__(self, parquet_file, value_column='value', window_size=10):
        self.parquet_file = parquet_file
        self.value_column = value_column
        self.window_size = window_size
        self.table = pq.read_table(parquet_file, columns=[value_column])
        chunked_array = self.table.column(value_column)
        self.values = np.concatenate([chunk.to_numpy() for chunk in chunked_array.chunks])

        self.length = len(self.values) - window_size

    def __len__(self):
        return self.length

    def __getitem__(self, idx):
        x = torch.tensor(self.values[idx:idx + self.window_size], dtype=torch.float32)
        y = torch.tensor(self.values[idx + self.window_size], dtype=torch.float32)
        return x, y



class LSTMForecast(nn.Module):
    def __init__(self, input_size=1, hidden_size=64, num_layers=2):
        super(LSTMForecast, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.linear = nn.Linear(hidden_size, 1)

    def forward(self, x):
        out, _ = self.lstm(x.unsqueeze(-1))  # (batch, seq_len, 1)
        out = self.linear(out[:, -1, :])     # Take last timestep
        return out.squeeze()


In [12]:
from torch.utils.data import DataLoader
import torch.optim as optim
from tqdm import tqdm

def train_model(dataset, model, epochs=10, batch_size=32, lr=1e-3):
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)

    model.train()
    for epoch in range(epochs):
        losses = []
        for x, y in tqdm(dataloader, desc=f"Epoch {epoch+1}/{epochs}"):
            optimizer.zero_grad()
            preds = model(x)
            loss = criterion(preds, y)
            loss.backward()
            optimizer.step()
            losses.append(loss.item())
        print(f"Epoch {epoch+1} Loss: {sum(losses)/len(losses):.4f}")

parquet_path = 'data/train.parquet'
dataset = TimeSeriesParquetDataset(parquet_path, value_column='is_anomaly', window_size=20)
model = LSTMForecast(input_size=1, hidden_size=64, num_layers=2)

train_model(dataset, model, epochs=10, batch_size=64, lr=0.001)


Epoch 1/10:  31%|███       | 70813/230130 [36:10<1:21:22, 32.63it/s]


KeyboardInterrupt: 